In [1]:
%pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [76]:
import random
import socket
import numpy as np
import scipy
import librosa
import io
from pydub import AudioSegment
import tensorflow as tf
from tensorflow.keras.models import load_model
import sklearn.preprocessing

In [70]:
model = load_model('./model-esp-32-01.h5')

2023-05-19 23:46:44.870416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-19 23:46:44.871891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-19 23:46:44.873578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [71]:
def calc_mfcc(x):
  return librosa.feature.mfcc(y=x / 32767, sr=16000, hop_length=128, htk=True)

In [72]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('', 1234))

In [75]:
wav_test = []
ds = []
while True:
  message, address = server_socket.recvfrom(1000)
  data = list(message)
  assert len(data) == 1000
  wav_test.extend(data)
  ds.extend(data)
  if len(ds) == 32000:
    file = AudioSegment.from_raw(io.BytesIO(bytes(ds)), sample_width=2, frame_rate=16000, channels=1)
    audio = np.array(file.get_array_of_samples())
    mfcc = calc_mfcc(audio)
    mfcc = sklearn.preprocessing.scale(mfcc)
    print(mfcc.shape)
    ds = []
    pred = model.predict(np.array([mfcc]), verbose=None)
    pos = np.argmax(pred[0])
    print(pos, int(pred[0][pos] * 100 * 100) / 100)

(20, 126)


2023-05-19 23:47:26.952083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-19 23:47:26.953518: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-19 23:47:26.954874: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

0 99.93
(20, 126)
0 94.24
(20, 126)
1 77.43
(20, 126)
0 96.78
(20, 126)
0 98.43
(20, 126)
2 46.08
(20, 126)
0 98.89
(20, 126)
0 99.73
(20, 126)
1 93.47
(20, 126)
0 99.88
(20, 126)
0 99.8
(20, 126)
1 85.7
(20, 126)
0 99.75
(20, 126)
0 97.58
(20, 126)
1 91.15
(20, 126)
0 99.32
(20, 126)
0 99.69
(20, 126)
0 99.99
(20, 126)
0 82.71
(20, 126)
0 81.35
(20, 126)
0 99.93
(20, 126)
0 85.45
(20, 126)
0 99.39
(20, 126)
0 97.44
(20, 126)
0 99.28
(20, 126)
0 93.5
(20, 126)
0 87.21
(20, 126)
0 89.21
(20, 126)
0 99.9
(20, 126)
0 99.73
(20, 126)
0 99.97
(20, 126)
0 99.24
(20, 126)
0 99.38
(20, 126)
0 99.99
(20, 126)
0 83.07
(20, 126)
0 99.94
(20, 126)
0 76.96
(20, 126)
0 99.99
(20, 126)
0 99.89
(20, 126)
2 48.74
(20, 126)
0 99.88
(20, 126)
0 99.99
(20, 126)
0 99.98
(20, 126)
0 92.75
(20, 126)
0 87.59
(20, 126)
0 99.52
(20, 126)
0 98.86
(20, 126)
2 99.91
(20, 126)
0 98.45
(20, 126)
0 93.97
(20, 126)
1 96.56
(20, 126)
0 47.23
(20, 126)
0 99.85
(20, 126)
0 79.72
(20, 126)
0 99.95
(20, 126)
1 95.01
(20, 1

KeyboardInterrupt: 

In [55]:
len(wav_test)

1183000

In [58]:
s = io.BytesIO(bytes(wav_test))
audio = AudioSegment.from_raw(s, sample_width=2, frame_rate=16000, channels=1).export('silent-01.wav', format='wav')

array([  512,   512,   768, ..., -1792, -1792, -1792], dtype=int16)

In [29]:
len(ds)

0